In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
import pandas as pd

df = pd.read_csv('/content/drive/My Drive/Online Retail.csv')
print(df.columns)
print('done read dataset')

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')
done read dataset


In [5]:
products = df[['StockCode','Description']].copy()
products.head()
products = products.groupby(products['StockCode']).filter(lambda x : x.notnull().values.all())
print('done creating product dictionary')

products.head()

import pickle as pk
userRfmTable = pk.load(open('/content/drive/My Drive/userRfmTable (1).pk','rb'))

done creating product dictionary


In [0]:
userRfmTable

productsPerUser = df

In [7]:
productsPerUser = productsPerUser[productsPerUser['Quantity'] > 0]
productsPerUser = productsPerUser[productsPerUser['UnitPrice'] > 0]
print(productsPerUser.head())
productsPerUser = productsPerUser.groupby(['CustomerID','StockCode','InvoiceNo'])['Quantity'].sum().reset_index()
productsPerUser

  InvoiceNo StockCode  ... CustomerID         Country
0    536365    85123A  ...    17850.0  United Kingdom
1    536365     71053  ...    17850.0  United Kingdom
2    536365    84406B  ...    17850.0  United Kingdom
3    536365    84029G  ...    17850.0  United Kingdom
4    536365    84029E  ...    17850.0  United Kingdom

[5 rows x 8 columns]


,CustomerID,StockCode,InvoiceNo,Quantity
0,12346.0,23166,541431,74215
1,12347.0,16008,549222,24
2,12347.0,17021,556201,36
3,12347.0,20665,549222,6
4,12347.0,20719,542237,10
5,12347.0,20719,562032,10
6,12347.0,20719,573511,10
7,12347.0,20719,581180,10
8,12347.0,20780,537626,12
9,12347.0,20782,537626,6


In [8]:
productsPerUser['StockCode'].sample(n=1)

userClusterStockTable = userRfmTable.merge(productsPerUser, on='CustomerID',how='left')
#userRfmTable
userClusterStockTable['cluster'].value_counts()

0    347180
1     30038
3      8116
2      2507
Name: cluster, dtype: int64

In [9]:
print(userClusterStockTable.head())
userClusterStockTable = userClusterStockTable[['StockCode','cluster','clusterSub','InvoiceNo','Quantity']]#[['Quantity']].sum().reset_index()
print(userClusterStockTable.describe())
testCluster = 0
itemTest = '23194'

invoices = userClusterStockTable[userClusterStockTable['cluster']==testCluster]['InvoiceNo']

recomendedItems = userClusterStockTable[userClusterStockTable['InvoiceNo'].isin(invoices)].groupby(['StockCode'])['Quantity'].sum().sort_values(ascending=False)#.drop(itemTest)
print(recomendedItems.head())
products= products.drop_duplicates()
#print(products[products['StockCode']==itemTest])
products[products['StockCode'].isin(recomendedItems.head().index)].head()
print(products[products['StockCode'].isin(recomendedItems.head().index)].head())
import pickle as pk

pk.dump(userClusterStockTable,open('userClusterStockTable.pk','wb'))
pk.dump(products,open('products.pk','wb'))

   CustomerID  recency  frequency  ...  StockCode  InvoiceNo  Quantity
0     12346.0      325          1  ...      23166     541431     74215
1     12347.0        2        182  ...      16008     549222        24
2     12347.0        2        182  ...      17021     556201        36
3     12347.0        2        182  ...      20665     549222         6
4     12347.0        2        182  ...      20719     542237        10

[5 rows x 9 columns]
             cluster     clusterSub       Quantity
count  387841.000000  347180.000000  387841.000000
mean        0.153156       1.269670      13.324563
std         0.517865       0.679171     181.672806
min         0.000000       0.000000       1.000000
25%         0.000000       1.000000       2.000000
50%         0.000000       1.000000       6.000000
75%         0.000000       2.000000      12.000000
max         3.000000       2.000000   80995.000000
StockCode
84077     40680
22197     27248
84879     27111
85099B    25528
22616     24107
Nam

In [32]:
products.head()
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel

tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(products['Description'])

cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)
tmp_product = products
#tmp_product['tfidf'] = tf.fit_transform(products['Description'])
#tmp_product = products[products['StockCode'].isin(recomendedItems.head().index)]
#tmp_product['tfidf']=tf.fit_transform(tmp_product)
tmp_product
print(tf.fit_transform(products['Description']).shape)
print(tf.fit_transform(products['Description']))
print(products.head())

(3688, 13046)
<bound method _cs_matrix.toarray of <3688x13046 sparse matrix of type '<class 'numpy.float64'>'
	with 31698 stored elements in Compressed Sparse Row format>>
  StockCode  ...                                              tfidf
0    85123A  ...    (0, 5816)\t0.32494653114901045\n  (0, 5655)\...
1     71053  ...    (0, 5816)\t0.32494653114901045\n  (0, 5655)\...
5     22752  ...    (0, 5816)\t0.32494653114901045\n  (0, 5655)\...
6     21730  ...    (0, 5816)\t0.32494653114901045\n  (0, 5655)\...
9     84879  ...    (0, 5816)\t0.32494653114901045\n  (0, 5655)\...

[5 rows x 3 columns]
